In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D

# 1. Load Data
df = pd.read_csv("D:/Ngoding/ASSAY.csv")
print(df)

# 2. Data Cleaning
df.dropna(inplace=True)  # Hapus nilai kosong
df = df[(df['depth_from'] < df['depth_to'])]  # Pastikan kedalaman valid

# 2.1 Pengatasan Outlier
cols = ['ni', 'fe', 'mgo'] 
for col in cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

# 3. Domaining Berdasarkan Kriteria
conditions = [(df['fe'] > 40) & (df['ni'] < 1),(df['mgo'] > 10) & (df['ni'] > 1),(df['ni'] < 0.3)]
zones = ['Limonit', 'Saprolit', 'Bedrock']
df['zone'] = np.select(conditions, zones, default='Transisi')

# 4. Visualisasi Distribusi Kadar Nikel
plt.figure(figsize=(10, 5))
sns.histplot(df['ni'], bins=30, kde=True)
plt.title("Distribusi Kadar Nikel")
plt.xlabel("Kadar Ni (%)")
plt.ylabel("Frekuensi")
plt.show()

plt.figure(figsize=(8, 5))
sns.boxplot(x='zone', y='ni', data=df)
plt.title("Boxplot Kadar Ni per Zona")
plt.xlabel("Zona Laterit")
plt.ylabel("Kadar Ni (%)")
plt.show()

# 5. Pemodelan Domaining
## K-means Clustering
X = df[['ni', 'fe', 'mgo']]
kmeans = KMeans(n_clusters=3, random_state=42).fit(X)
df['cluster'] = kmeans.labels_

plt.figure(figsize=(8, 6))
sns.scatterplot(x=df['fe'], y=df['ni'], hue=df['cluster'], palette='viridis')
plt.xlabel("Kadar Fe (%)")
plt.ylabel("Kadar Ni (%)")
plt.title("Clustering Laterit dengan K-means")
plt.show()

# 6. Interpolasi dengan IDW
xi = np.linspace(df['depth_from'].min(), df['depth_from'].max(), 100)
yi = np.linspace(df['ni'].min(), df['ni'].max(), 100)
X_grid, Y_grid = np.meshgrid(xi, yi)
Z_grid = griddata((df['depth_from'], df['ni']), df['fe'], (X_grid, Y_grid), method='cubic')

plt.figure(figsize=(10, 6))
plt.contourf(X_grid, Y_grid, Z_grid, cmap='coolwarm')
plt.colorbar(label='Kadar Fe (%)')
plt.xlabel("Kedalaman (m)")
plt.ylabel("Kadar Ni (%)")
plt.title("Interpolasi Kadar Fe dengan IDW")
plt.show()

# 7. 3D Plotting untuk Visualisasi Domain
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['depth_from'], df['ni'], df['fe'], c=df['cluster'], cmap='viridis', alpha=0.7)
ax.set_xlabel("Kedalaman (m)")
ax.set_ylabel("Kadar Ni (%)")
ax.set_zlabel("Kadar Fe (%)")
ax.set_title("3D Plot Profil Nikel Laterit")
plt.show()
